In [1]:
import numpy as np
from cvxopt import matrix,solvers
import math as m
import time

We have done the production planning application and defined the A,B and C matrix in standard form(max C^T.x such 
that Ax<=b, modify those accordingly for different examples,(for max C^T.x with constraint AX <=B, C array values should be written in negative, if we have to maximize function )

In [2]:
A=np.array([[100,60,120,70],[-2,-1,0,0],[0,0,-3,-2],[1,1,0,0],[0,0,1,1]]).astype(np.float64)
B=np.array([990,-7,-16,10,6]).astype(np.float64)
C=np.array([-40,-20,-30,-10]).astype(np.float64)
solution=[]
iterations=[]

In below blocks, we have done preprocessing to include constraint that all variables are greater than zero,
and then we have written code which will branch the variable and bound it by floor and ceil function, we will take variable
which has highest role in optimum function, and then we will run algorithm

In [3]:
def w(sol):
    weigh=np.abs(C)
    for a in range(len(C)):
        if m.floor(sol[a])==m.ceil(sol[a]):
            weigh[a]=0
    return weigh

In [4]:
def preprocessing_BAB(c,a,b) :
    Nc=np.shape(a)[0]
    Nv=np.shape(a)[1]
    b=np.hstack((b,(np.zeros(Nv))))
    I=np.eye(Nv)
    a=np.append(a,-I,axis=0)
    c=matrix(c)
    b=matrix(b)
    a=matrix(a)
    return c,a,b

Running the branch and bound algorithm , and showing thr results below

In [5]:
def BAB(A,B,C):
    C,A,B=preprocessing_BAB(C,A,B)
    sol=solvers.lp(C,A,B);
    iterations.append(1)
    
    if sol['x'] is None:
        return 0
    sol['x']=np.round(sol['x'],3)
    print(sol['x'])
    
    weight=w(sol['x'])
    i=int(np.argmax(weight))
    d=[m.floor(sol['x'][i]),m.ceil(sol['x'][i])]
    
    if d[0]==d[1]:
        if(np.sum(weight)==0):
            solution.append(sol['x'])
        return sol['x']
    
    
        
    # ceil 
    z=np.zeros(len(C))
    z[i]=-1
    A1=np.vstack([A,z])
    B1=np.append(B,-d[1])
    sol1=BAB(A1,B1,C)
    # checkint(sol1)

    #floor
    z=np.zeros(len(C))
    z[i]=1
    A2=np.vstack([A,z])
    B2=np.append(B,d[0])
    sol2=BAB(A2,B2,C)
    #checkint(sol2)
    
    

In [6]:
start_time=time.time()
BAB(A,B,C)
            

     pcost       dcost       gap    pres   dres   k/t
 0: -2.8002e+02 -3.7933e+03  1e+02  7e-03  1e+01  1e+00
 1: -2.9795e+02 -4.8734e+02  6e+00  4e-04  6e-01  3e-01
 2: -2.9995e+02 -3.0359e+02  1e-01  7e-06  1e-02  4e-03
 3: -3.0000e+02 -3.0004e+02  1e-03  7e-08  1e-04  4e-05
 4: -3.0000e+02 -3.0000e+02  1e-05  7e-10  1e-06  4e-07
 5: -3.0000e+02 -3.0000e+02  1e-07  7e-12  1e-08  4e-09
Optimal solution found.
[[3.5  ]
 [0.   ]
 [5.333]
 [0.   ]]
     pcost       dcost       gap    pres   dres   k/t
 0: -2.8528e+02 -3.0504e+03  2e+02  9e-03  9e+00  1e+00
 1: -2.9392e+02 -9.4571e+02  3e+01  2e-03  2e+00  1e+00
 2: -2.7679e+02 -1.8252e+03  4e+02  5e-03  5e+00  8e+00
 3: -3.0363e+02 -9.3429e+02  3e+02  2e-03  2e+00  2e+01
 4: -2.8319e+02 -5.5379e+02  2e+02  9e-04  9e-01  1e+01
 5: -2.8249e+02  3.8220e+02  2e+04  1e-03  1e+00  1e+03
 6: -2.8248e+02  1.0040e+05  2e+06  1e-03  1e+00  1e+05
 7: -2.8248e+02  1.0102e+07  2e+08  1e-03  1e+00  1e+07
 8: -2.8248e+02  1.0103e+09  2e+10  1e-03  1e+0

In [7]:
objective=np.zeros(len(solution))
for j in range(len(solution)):
    x=solution[j]
    objective[j]=np.matmul(C.T,x)
    
optimal=np.argmin(objective)
x=solution[optimal]
print(x,objective[optimal])

[[3.]
 [1.]
 [4.]
 [2.]] -280.0


In [8]:
print(len(iterations))
print(np.round(time.time()-start_time,3))

9
0.156
